# text's feature engineering: turn unstructure to  structure

https://www.youtube.com/watch?v=5uNQN5XphM4&list=PL1f_B9coMEeCpkBHXNJBPoVY5LcbDMHvk&index=7&t=0s

In [1]:
import re
import pandas as pd
import numpy as np
import pickle
import os

In [2]:
## turn back to main directory
os.chdir("../")
os.getcwd()

'C:\\Users\\user\\Desktop\\Resume\\V2.5\\v2-5-nlp\\Part1'

In [3]:
df = pd.read_csv('data/article_preprocessed.csv')

In [4]:
# 路徑更改os.chdir()

#os.chdir("Part1/answer")
#os.getcwd()

In [5]:
## load 'article_cutted'

with open("data/article_cutted", "rb") as file:
    sentences = pickle.load(file)

## define y (push > boo)

In [6]:
## drop data

#排除中間地帶 Threshold設20 區分此文為推或噓
diff_threshold = 20

#如果大於threshold 儲存此篇推噓文
df = df[abs(df['push']-df['boo']) > diff_threshold].copy()

In [7]:
## define y


#推文筆噓文多: 1 推文筆噓文少: 0
df['type'] = np.clip(df['push']-df['boo'], 0, 1)
df = df.reset_index(drop=True)

In [8]:
df['type'].value_counts() #推: 17318 噓: 1134

1    17318
0     1134
Name: type, dtype: int64

## 1.simple feature  如何產生特徵 ?  可以產生甚麼特徵 ? 以本文為例...


- 基本資訊
- 字數、標點符號數、中文字比例、“問號”佔字數比例、...
- 關鍵字
- 有 / 無某關鍵字、關鍵字出現次數 / 佔比、...
- 關鍵字詞組
- 名詞、正面情緒詞、...
- 文本情緒分數



 ## 2. 以下3種特徵範例 : 
- 字數
- 標點符號數
- “問號”佔字數比例


- ## 正規表示式 Regular Expression:　[a-zA-Z0-9]　[\u4e00-\u9fff]

參考資料：https://poychang.github.io/note-regular-expression/

         https://www.itread01.com/content/1513168876.html

[a-zA-Z0-9]	含數字或字母之字串<br>

[/u4e00-/u9fa5] 匹配中文字符的正則表達式


In [9]:
## word count
## http://blog.csdn.net/gatieme/article/details/43235791 (中文正則表達式)
df['word_count'] = df['content'].str.count('[a-zA-Z0-9]+') + df['content'].str.count('[\u4e00-\u9fff]')
df['word_count']

0        175
1        145
2        393
3        295
4         41
        ... 
18447     99
18448    253
18449    867
18450    230
18451     60
Name: word_count, Length: 18452, dtype: int64

In [10]:
#給下面的程式做比對雜七雜八的標點

df["content"]

0        反核覺青現在最強招式就是 不然燃料棒放你家 肥宅我覺得 把燃料棒放到其他國家不就好了 一定會...
1        品牌電腦主打的就是方便 機子有問題，保固內免費幫你處理到好 不用還要在那邊自己除錯浪費一堆時...
2        反核人士最愛靠妖 核電廠蓋你家好不好，我當然說好 核廢料放我家好不好，我也ok 放在地下室就...
3        各位魯哥魯姐好 魯弟有個疑問 一休和尚卡通版大家多少都看過 可是長大後的版本沒有演到(是長大...
4        如標題， 今天去逛才看到的，如下圖所示:  位置在西屯區漢口路二段118號。 少了一個可以看...
                               ...                        
18447    才華 很會讀書的台清交肥宅 不夠有才華嗎?? 很會寫程式的竹科宅 不夠有才華嗎???? 靠妖...
18448    交往多年的女友抽到國外工作簽 才剛去不到一個月 就被室友猛烈追求 然後就跑了 幹幹幹幹幹幹幹...
18449    如標題 我一直不懂活佛的定義是什麼？？ 達賴也是人生父母養，為何西藏人隨變講個鄉下小孩，說他...
18450    大家都知道台灣人有多健忘， 之前鬧得滿城風雨的林奕含事件， 好像也只持續了一兩週， 然後陳星...
18451    跟新認識的洋妞約周六吃飯 在西門町 要吃什麼才能代表台灣 又讓人印象深刻呢 我是覺得什麼阿雜...
Name: content, Length: 18452, dtype: object

In [11]:
## punctuation count

# 計算這些雜七雜八的標點
df['punctuation'] = df['content'].str.replace('[\w\s]', '')
print(df['punctuation'])
print("")

# 計算雜七雜八標點的長度
df['punctuation_count'] = df['punctuation'].str.len()
print(df['punctuation_count'])

0                                                         
1                                                     ，？，，
2                                        ，，，?，，????.??，?，，
3                                          ()..()????...??
4                                                     ，，:。
                               ...                        
18447                                          ???????????
18448                                      ...............
18449    ？？，，，？，？，，.，.，？，？？？，？？？，？，，？，？，，.，？？？，.，，？，.？？...
18450                                ，，，。，，，。，，，？？，，，，，，，。
18451                                                 ...?
Name: punctuation, Length: 18452, dtype: object

0         0
1         4
2        17
3        15
4         4
         ..
18447    11
18448    15
18449    99
18450    21
18451     4
Name: punctuation_count, Length: 18452, dtype: int64


In [12]:
## question mark count 問號佔句子總數
df['question_count'] = df['punctuation'].str.count('[?？]')
df['question_count']

0         0
1         1
2         8
3         6
4         0
         ..
18447    11
18448     0
18449    43
18450     2
18451     1
Name: question_count, Length: 18452, dtype: int64

In [13]:
## drop punctuation column 將雜七雜八標點符號drop掉
df = df.drop(['punctuation'],axis=1)
df #drop掉的檔案

,id,title,content,date,push,boo,idx,type,word_count,punctuation_count,question_count
0,M.1502379457.A.440,[問卦] 核電燃料棒 跟其他國家買空間放不就好了,反核覺青現在最強招式就是 不然燃料棒放你家 肥宅我覺得 把燃料棒放到其他國家不就好了 一定會...,Thu Aug 10 23:37:34 2017,37,5,32,1,175,0,0
1,M.1502378858.A.109,[問卦] 為什麼很多人喜歡酸品牌電腦？,品牌電腦主打的就是方便 機子有問題，保固內免費幫你處理到好 不用還要在那邊自己除錯浪費一堆時...,Thu Aug 10 23:27:35 2017,26,3,39,1,145,4,1
2,M.1502378200.A.269,[問卦] 火力發電廠蓋你家好不好?,反核人士最愛靠妖 核電廠蓋你家好不好，我當然說好 核廢料放我家好不好，我也ok 放在地下室就...,Thu Aug 10 23:16:37 2017,66,13,67,1,393,17,8
3,M.1502377612.A.CC8,[問卦] 一休大師最後是跟誰在一起??,各位魯哥魯姐好 魯弟有個疑問 一休和尚卡通版大家多少都看過 可是長大後的版本沒有演到(是長大...,Thu Aug 10 23:06:49 2017,36,4,79,1,295,15,6
4,M.1502377278.A.803,[問卦] 台中諾貝爾倒了一家,如標題， 今天去逛才看到的，如下圖所示: 位置在西屯區漢口路二段118號。 少了一個可以看...,Thu Aug 10 23:01:15 2017,51,3,92,1,41,4,0
...,...,...,...,...,...,...,...,...,...,...,...
18447,M.1505407607.A.D08,"[問卦] 女生的""男生要有才華""到底是啥意思???",才華 很會讀書的台清交肥宅 不夠有才華嗎?? 很會寫程式的竹科宅 不夠有才華嗎???? 靠妖...,Fri Sep 15 00:46:45 2017,44,3,252150,1,99,11,11
18448,M.1505407146.A.67A,[爆卦] 幹幹幹幹幹,交往多年的女友抽到國外工作簽 才剛去不到一個月 就被室友猛烈追求 然後就跑了 幹幹幹幹幹幹幹...,Fri Sep 15 00:39:04 2017,920,84,252161,1,253,15,0
18449,M.1505406664.A.4D0,[問卦] 為何達賴可以是活佛，妙禪不能是活佛,如標題 我一直不懂活佛的定義是什麼？？ 達賴也是人生父母養，為何西藏人隨變講個鄉下小孩，說他...,Fri Sep 15 00:31:02 2017,75,108,252173,0,867,99,43
18450,M.1505406079.A.1AF,[問卦] 這次反妙禪seafood的熱潮能持續幾天？,大家都知道台灣人有多健忘， 之前鬧得滿城風雨的林奕含事件， 好像也只持續了一兩週， 然後陳星...,Fri Sep 15 00:21:17 2017,152,9,252195,1,230,21,2


In [14]:
# iloc  :  Pandas 透過使用中括號 [] 與 .iloc 可以很靈活地從 data frame 中選擇想要的元素。

#可以看到整理完的資料及處理掉的雜七雜八資料分布
df.iloc[:8 , -5:]

,idx,type,word_count,punctuation_count,question_count
0,32,1,175,0,0
1,39,1,145,4,1
2,67,1,393,17,8
3,79,1,295,15,6
4,92,1,41,4,0
5,100,1,80,4,3
6,104,1,1362,128,3
7,106,0,71,0,0


In [15]:
## compute correlation (相關係數)
df.iloc[:, -4:].corr()

,type,word_count,punctuation_count,question_count
type,1.000000,-0.045100,-0.024124,-0.056966
word_count,-0.045100,1.000000,0.738419,0.534900
punctuation_count,-0.024124,0.738419,1.000000,0.342150
question_count,-0.056966,0.534900,0.342150,1.000000


## bag of words (關鍵次數出現次數)

In [16]:
# CountVectorizer : 可以自動把文字內容斷好詞並轉換
from sklearn.feature_extraction.text import CountVectorizer

In [17]:
CountVectorizer()

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

- join用法 : https://www.runoob.com/python3/python3-string-join.html

In [18]:
## define transformer (轉換器)
vectorizer = CountVectorizer()
count = vectorizer.fit_transform([' '.join(x) for x in sentences])

In [19]:
count

<252229x372654 sparse matrix of type '<class 'numpy.int64'>'
	with 7700068 stored elements in Compressed Sparse Row format>

In [20]:
## save data as pickle format
with open("data/article_count", "wb") as file:
    pickle.dump([vectorizer, count], file)

- ### select top 10 frequency of words


#### Practice :  create a dictionary: id as key ; word as values


In [21]:
## 建dictionary, create a dictionary: id as key ; word as values
id2word = {v:k for k, v in vectorizer.vocabulary_.items()}
id2word

{359734: '韓瑜',
 88964: '協志',
 81264: '前妻',
 124193: '女演員',
 101120: '周子',
 32794: 'twice',
 112012: '團裡裡面',
 95899: '台灣',
 72855: '兩個',
 304815: '要當',
 341779: '鄉民',
 279966: '老婆',
 52258: '五樓',
 84763: '勇氣',
 7726: 'dear',
 1315: 'all',
 330651: '逢甲',
 259978: '碟仙',
 252403: '發生',
 217487: '民國',
 39844: '七十五年',
 40803: '三月中',
 50839: '事情',
 37544: '一堆',
 119192: '大學生',
 155407: '後發',
 3246: 'bbs',
 165238: '成功',
 360505: '預測',
 113080: '地震',
 138258: '小弟',
 360539: '預言',
 341703: '都還沒',
 77977: '出生',
 155475: '後面',
 38557: '一百',
 37283: '一十六年',
 73070: '兩岸',
 274242: '統一',
 137003: '對岸',
 164759: '應該',
 44570: '不用',
 257519: '真的',
 129416: '存在',
 329346: '這種',
 64673: '倒底',
 301764: '被統',
 258736: '知道',
 316819: '資料庫',
 252273: '發文',
 190624: '日期',
 324826: '輕鬆',
 184605: '改變',
 174517: '拍照',
 240159: '狀況',
 257520: '真的假',
 198934: '有沒有',
 262760: '科學',
 274758: '經驗',
 222749: '法則',
 259587: '破解',
 311967: '謠言',
 257318: '真實',
 73627: '八卦',
 193248: '晚上',
 360458: '預備',
 362378: '風雲',

In [22]:
## columnwise sum: words frequency 個欄位總和
sum_ = np.array(count.sum(axis=0))[0]
sum_

array([ 1,  1,  1, ..., 17,  1,  1], dtype=int64)

In [23]:
## 挑出前10的wordID / top 10 frequency's wordID
most_sum_id = sum_.argsort()[::-1][:10].tolist()
print(most_sum_id)
type(most_sum_id)


[73627, 198934, 95899, 37001, 243708, 258736, 257519, 305714, 256024, 283981]


list

In [24]:
## print出 top10 : print top 10 frequency's words
features = [id2word[i] for i in most_sum_id]
features

['八卦', '有沒有', '台灣', '一個', '現在', '知道', '真的', '覺得', '看到', '肥宅']

In [27]:
df.idx

0            32
1            39
2            67
3            79
4            92
          ...  
18447    252150
18448    252161
18449    252173
18450    252195
18451    252199
Name: idx, Length: 18452, dtype: int64

In [28]:
## 轉換出 Data table : print the data

data = pd.DataFrame(count[df.idx,:][:,most_sum_id].toarray(), columns=features)
data

,八卦,有沒有,台灣,一個,現在,知道,真的,覺得,看到,肥宅
0,1,1,1,0,2,0,0,1,0,1
1,1,1,0,0,0,0,0,0,0,0
2,0,0,0,1,1,0,0,1,0,0
3,1,2,0,2,0,2,0,0,0,0
4,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...
18447,0,0,0,0,0,0,0,0,0,0
18448,1,0,0,0,0,0,0,0,0,1
18449,0,1,1,8,1,1,0,0,0,0
18450,0,0,2,0,0,1,0,0,0,0


In [29]:
# check data
df.iloc[0].content

'反核覺青現在最強招式就是 不然燃料棒放你家 肥宅我覺得 把燃料棒放到其他國家不就好了 一定會有缺錢的國家 台灣塞錢給他們 買他們國家的空間放 一來燃料棒問題解決 核電重啟 台灣缺點問題解決 大家有冷氣吹 台積電不出走 繼續救台灣 二來有買賣就有貪污空間 政客也有賺頭 不會像現在沒糖吃該該叫 送錢出去 邦交國搞不好也會多幾個 簡直是雙贏 核電燃料棒 跟其他國家買空間放不就好了 有沒有相關八卦 '

In [30]:
type(df.type)

pandas.core.series.Series

In [31]:
## compute correlation
data = pd.concat([df.type, data], axis=1)
data.corr()

,type,八卦,有沒有,台灣,一個,現在,知道,真的,覺得,看到,肥宅
type,1.000000,-0.056491,0.000898,-0.048109,-0.032585,-0.018120,-0.019432,-0.021593,-0.012116,-0.048172,-0.013955
八卦,-0.056491,1.000000,0.431786,-0.001901,0.062416,0.052670,0.091136,0.084516,0.066589,0.066047,0.077210
有沒有,0.000898,0.431786,1.000000,0.032712,0.094241,0.105734,0.144891,0.105919,0.095098,0.089291,0.043573
台灣,-0.048109,-0.001901,0.032712,1.000000,0.194281,0.188974,0.166716,0.162471,0.125782,0.137087,-0.033735
一個,-0.032585,0.062416,0.094241,0.194281,1.000000,0.400985,0.523627,0.398092,0.370977,0.327872,0.026330
現在,-0.018120,0.052670,0.105734,0.188974,0.400985,1.000000,0.418126,0.348470,0.306070,0.230148,0.009126
知道,-0.019432,0.091136,0.144891,0.166716,0.523627,0.418126,1.000000,0.486961,0.391520,0.336525,0.037975
真的,-0.021593,0.084516,0.105919,0.162471,0.398092,0.348470,0.486961,1.000000,0.461401,0.331108,0.039239
覺得,-0.012116,0.066589,0.095098,0.125782,0.370977,0.306070,0.391520,0.461401,1.000000,0.270267,0.026513
看到,-0.048172,0.066047,0.089291,0.137087,0.327872,0.230148,0.336525,0.331108,0.270267,1.000000,0.020214


# TF-IDF

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [33]:
## define transformer (轉換器)
vectorizer = TfidfVectorizer(norm=None) ## do not do normalize
tfidf = vectorizer.fit_transform([' '.join(x) for x in sentences])

In [34]:
## save data as pickle format
with open("data/article_tfidf", "wb") as file:
    pickle.dump([vectorizer, tfidf], file)

### select top 10 average tf-idf of words

In [35]:
## create a dictionary: id as key ; word as values
id2word = {v:k for k, v in vectorizer.vocabulary_.items()}

In [36]:
## columnwise average: words tf-idf
avg = tfidf.sum(axis=0) / (tfidf!=0).sum(axis=0)

## 設threshold 20,以下為零 因為出現太少 tf-idf會被放的太大
# set df < 20 as 0
avg[(tfidf!=0).sum(axis=0)<20] = 0

In [39]:
avg = np.array(avg)[0]

In [40]:
## top 10 tfidf's wordID 抽出欄位
most_avg_id = avg.argsort()[::-1][:10].tolist()
#np.array(most_avg_id)[0,:5]
most_avg_id[:][:5]

[90835, 325364, 157970, 263428, 357411]

In [41]:
## 用id to word 找出關鍵字為何 
# print top 10 tf-idf's words

features = [id2word[i] for i in most_avg_id]
features

['原告', '轉帳', '忍術', '稅後', '震度', 'charlie', '中山路', 'united', '二段', '支出']

In [42]:
## print the data
data = pd.DataFrame(count[df.idx,:][:,most_avg_id].toarray(), columns=features)

#data = pd.DataFrame(tfidf[df.idx.as_matrix(),:][:,most_avg_id].toarray(), columns=features)
data[:5]

,原告,轉帳,忍術,稅後,震度,charlie,中山路,united,二段,支出
0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1,0


In [43]:
## 看correlation (compute correlation)
data = pd.concat([df.type, data], axis=1)
data.corr()

,type,原告,轉帳,忍術,稅後,震度,charlie,中山路,united,二段,支出
type,1.000000,-0.004499,0.002090,0.002664,0.001970,0.007263,0.002013,0.001974,0.003889,0.001882,0.002063
原告,-0.004499,1.000000,-0.000221,-0.000177,-0.000238,-0.000483,-0.000134,-0.000131,-0.000258,-0.000132,-0.000226
轉帳,0.002090,-0.000221,1.000000,-0.000136,0.999764,-0.000370,-0.000102,-0.000100,-0.000198,-0.000101,0.999931
忍術,0.002664,-0.000177,-0.000136,1.000000,-0.000146,-0.000296,-0.000082,-0.000080,-0.000158,-0.000081,-0.000138
稅後,0.001970,-0.000238,0.999764,-0.000146,1.000000,-0.000397,-0.000110,-0.000108,-0.000213,-0.000109,0.999715
震度,0.007263,-0.000483,-0.000370,-0.000296,-0.000397,1.000000,-0.000223,-0.000219,-0.000431,-0.000221,-0.000377
charlie,0.002013,-0.000134,-0.000102,-0.000082,-0.000110,-0.000223,1.000000,-0.000061,-0.000120,-0.000061,-0.000104
中山路,0.001974,-0.000131,-0.000100,-0.000080,-0.000108,-0.000219,-0.000061,1.000000,-0.000117,0.999799,-0.000102
united,0.003889,-0.000258,-0.000198,-0.000158,-0.000213,-0.000431,-0.000120,-0.000117,1.000000,-0.000118,-0.000202
二段,0.001882,-0.000132,-0.000101,-0.000081,-0.000109,-0.000221,-0.000061,0.999799,-0.000118,1.000000,-0.000103


- # 補充其他的文意詞彙範例


   中文版LIWC詞典 : https://cliwc.weebly.com/
   
   Ehownet : 對文字定義(詞分類)  http://ehownet.iis.sinica.edu.tw/ehownet.php

![](LIWC.JPG)